### Import libraries

In [1]:
#!pip install geocoder
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import geocoder

### Get data

In [2]:
# Get Wikipedia data
wikipedia_page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(wikipedia_page,'xml')
get_table = soup.find('table')

# Get latitude and longitude data
df_latlng = pd.read_csv('https://cocl.us/Geospatial_data')

### Process data

In [3]:
# Initialize Postcode, Borough, Neighbourhood, Latitude, and Longitude
Postcode = []
Borough = []
Neighbourhood = []
Latitude = []
Longitude = []

# Populate Postcode, Borough, Neighbourhood, Latitude, and Longitude
for get_tr in get_table.find_all('tr'):
    
    index = 1
    tempPostcode = ''
    tempBorough = ''
    tempNeighbourhood = ''
    
    for get_td in get_tr.find_all('td'):
        if index == 1: 
            tempPostcode = str(get_td.text).strip()
        if index == 2: 
            tempBorough = str(get_td.text).strip()
        if index == 3: 
            tempNeighbourhood = str(get_td.text).strip()
        index += 1
    
    if tempBorough not in ['','Not assigned']:
        if tempNeighbourhood == 'Not assigned':
            tempNeighbourhood = tempBorough
        if tempPostcode in Postcode:
            Neighbourhood[Postcode.index(tempPostcode)] = Neighbourhood[Postcode.index(tempPostcode)] + ',' + tempNeighbourhood 
        if tempPostcode not in Postcode:
            Postcode.append(tempPostcode)
            Borough.append(tempBorough)
            Neighbourhood.append(tempNeighbourhood)

            df_latlng_index = df_latlng[df_latlng['Postal Code']==tempPostcode].index[0]
            Latitude.append(df_latlng.iloc[df_latlng_index]['Latitude'])
            Longitude.append(df_latlng.iloc[df_latlng_index]['Longitude'])

# Create dataframe
dict = {'Postcode':Postcode, 'Borough':Borough, 'Neighbourhood':Neighbourhood, 'Latitude':Latitude, 'Longitude':Longitude}
df = pd.DataFrame.from_dict(dict)
df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


### Get shape

In [4]:
df.shape

(103, 5)

### Import more libraries

In [5]:
#!pip install geopy
from geopy.geocoders import Nominatim
import json
from pandas.io.json import json_normalize
#!pip install folium
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

### Get Downtown Data

In [6]:
df_downtown = df[df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
df_downtown

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
5,M6G,Downtown Toronto,Christie,43.669542,-79.422564
6,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568
7,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752
8,M5K,Downtown Toronto,"Design Exchange,Toronto Dominion Centre",43.647177,-79.381576
9,M5L,Downtown Toronto,"Commerce Court,Victoria Hotel",43.648198,-79.379817


### Create downtown map

In [7]:
# get coordinates (using 1st entry)
downtown_lat = df_downtown.iloc[0]['Latitude']
downtown_lon = df_downtown.iloc[0]['Longitude']

# get map
downtown_map = folium.Map(location=[downtown_lat, downtown_lon], zoom_start=12)

# add markers to map
for lat, lng, label in zip(df_downtown['Latitude'], df_downtown['Longitude'], df_downtown['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(downtown_map)  

# display map    
downtown_map

### Define Foursquare Credentials and Version

In [8]:
CLIENT_ID = 'AC0MILWJ0GHICT52OB4DUREZHZRFGEQQU1BWLQMK11O0RIXC' # your Foursquare ID
CLIENT_SECRET = '05V4STGHSNHWDD52GQ2QOYHHQYHKC5L1U4ANPD5VQNNXU1SA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

### Explore downtown neighbourhoods

In [9]:
LIMIT = 100
radius = 1000

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
downtown_venues = getNearbyVenues(names=df_downtown['Neighbourhood'],
                                   latitudes=df_downtown['Latitude'],
                                   longitudes=df_downtown['Longitude']
                                  )

Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city
Church and Wellesley


### Analyze each neighborhood

In [11]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
downtown_onehot.shape

(1285, 205)

In [13]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000,0.01
1,Berczy Park,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.00,0.000000,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,"Cabbagetown,St. James Town",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,Central Bay Street,0.011494,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.011494,0.000000,0.000000,0.011494,0.000000,0.00
5,"Chinatown,Grange Park,Kensington Market",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.01,0.01,0.00,0.000000,0.060000,0.000000,0.040000,0.010000,0.000000,0.00
6,Christie,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
7,Church and Wellesley,0.011364,0.00,0.011364,0.011364,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.011364,0.011364,0.011364,0.000000,0.011364,0.00
8,"Commerce Court,Victoria Hotel",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.00
9,"Design Exchange,Toronto Dominion Centre",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.00


In [14]:
downtown_grouped.shape

(18, 205)

In [15]:
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.06
2           Steakhouse  0.04
3  American Restaurant  0.04
4      Thai Restaurant  0.04


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.06
2    Steakhouse  0.04
3        Bakery  0.04
4   Cheese Shop  0.04


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airport Terminal  0.14
3             Plane  0.07
4     Boat or Ferry  0.07


----Cabbagetown,St. James Town----
                venue  freq
0          Restaurant  0.08
1         Coffee Shop  0.08
2   Indian Restaurant  0.04
3              Bakery  0.04
4  Italian Restaurant  0.04


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.14
1                 Café  0.06
2   Italian Restaurant  0.05
3       Sandwich Place  

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Café,Coffee Shop,Steakhouse,American Restaurant,Thai Restaurant,Restaurant,Bar,Hotel,Cosmetics Shop,Gym
1,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Restaurant,Steakhouse,Café,Cheese Shop,Farmers Market,Bakery,Seafood Restaurant
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Terminal,Airport Service,Airport Lounge,Harbor / Marina,Airport Gate,Sculpture Garden,Boutique,Plane,Airport Food Court,Airport
3,"Cabbagetown,St. James Town",Restaurant,Coffee Shop,Pub,Flower Shop,Café,Indian Restaurant,Pizza Place,Italian Restaurant,Bakery,Bank
4,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Burger Joint,Chinese Restaurant,Bar,Japanese Restaurant,Bubble Tea Shop,Ice Cream Shop
5,"Chinatown,Grange Park,Kensington Market",Café,Bar,Vegetarian / Vegan Restaurant,Chinese Restaurant,Mexican Restaurant,Vietnamese Restaurant,Bakery,Dumpling Restaurant,Coffee Shop,Comfort Food Restaurant
6,Christie,Grocery Store,Café,Park,Baby Store,Italian Restaurant,Diner,Nightclub,Convenience Store,Restaurant,Coffee Shop
7,Church and Wellesley,Japanese Restaurant,Coffee Shop,Gay Bar,Sushi Restaurant,Burger Joint,Restaurant,Pub,Mediterranean Restaurant,Men's Store,Bubble Tea Shop
8,"Commerce Court,Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Deli / Bodega,Gastropub,Steakhouse,Italian Restaurant,Gym
9,"Design Exchange,Toronto Dominion Centre",Coffee Shop,Hotel,Café,American Restaurant,Italian Restaurant,Gym,Restaurant,Sports Bar,Gastropub,Deli / Bodega


### Cluster Neighborhoods

In [18]:
# set number of clusters
kclusters = 6

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 5, 3, 5, 2, 1, 4, 1, 2, 2])

In [19]:
downtown_merged = df_downtown

# add clustering labels
downtown_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

downtown_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,1,Coffee Shop,Café,Park,Bakery,Breakfast Spot,Pub,Theater,Mexican Restaurant,Dessert Shop,Restaurant
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,5,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Bar,Japanese Restaurant,Diner,Plaza,Sandwich Place,Pizza Place
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Coffee Shop,Café,Restaurant,Hotel,Clothing Store,Italian Restaurant,Park,Japanese Restaurant,Cocktail Bar,Cosmetics Shop
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,5,Coffee Shop,Cocktail Bar,Beer Bar,Restaurant,Steakhouse,Café,Cheese Shop,Farmers Market,Bakery,Seafood Restaurant
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Burger Joint,Chinese Restaurant,Bar,Japanese Restaurant,Bubble Tea Shop,Ice Cream Shop


In [20]:
# create map
map_clusters = folium.Map(location=[downtown_lat, downtown_lon], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighbourhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine clusters

In [21]:
# cluster 1
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Hotel,Seafood Restaurant,Cocktail Bar,Beer Bar,Cheese Shop,Pub,Bakery


In [22]:
# cluster 2
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Café,Park,Bakery,Breakfast Spot,Pub,Theater,Mexican Restaurant,Dessert Shop,Restaurant
5,Downtown Toronto,1,Grocery Store,Café,Park,Baby Store,Italian Restaurant,Diner,Nightclub,Convenience Store,Restaurant,Coffee Shop
7,Downtown Toronto,1,Coffee Shop,Hotel,Café,Pizza Place,Aquarium,Brewery,Italian Restaurant,Scenic Lookout,Sports Bar,Sushi Restaurant
11,Downtown Toronto,1,Café,Bar,Vegetarian / Vegan Restaurant,Chinese Restaurant,Mexican Restaurant,Vietnamese Restaurant,Bakery,Dumpling Restaurant,Coffee Shop,Comfort Food Restaurant


In [23]:
# cluster 3
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,2,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Burger Joint,Chinese Restaurant,Bar,Japanese Restaurant,Bubble Tea Shop,Ice Cream Shop
8,Downtown Toronto,2,Coffee Shop,Hotel,Café,American Restaurant,Italian Restaurant,Gym,Restaurant,Sports Bar,Gastropub,Deli / Bodega
9,Downtown Toronto,2,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Deli / Bodega,Gastropub,Steakhouse,Italian Restaurant,Gym
10,Downtown Toronto,2,Café,Bookstore,Coffee Shop,Japanese Restaurant,Bar,Bakery,Restaurant,Sushi Restaurant,Sandwich Place,Pub
12,Downtown Toronto,2,Airport Terminal,Airport Service,Airport Lounge,Harbor / Marina,Airport Gate,Sculpture Garden,Boutique,Plane,Airport Food Court,Airport
13,Downtown Toronto,2,Park,Playground,Trail,Deli / Bodega,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store


In [24]:
# cluster 4
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Hotel,Clothing Store,Italian Restaurant,Park,Japanese Restaurant,Cocktail Bar,Cosmetics Shop


In [25]:
# cluster 5
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,4,Café,Coffee Shop,Steakhouse,American Restaurant,Thai Restaurant,Restaurant,Bar,Hotel,Cosmetics Shop,Gym


In [26]:
# cluster 6
downtown_merged.loc[downtown_merged['Cluster Labels'] == 5, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,5,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Bar,Japanese Restaurant,Diner,Plaza,Sandwich Place,Pizza Place
3,Downtown Toronto,5,Coffee Shop,Cocktail Bar,Beer Bar,Restaurant,Steakhouse,Café,Cheese Shop,Farmers Market,Bakery,Seafood Restaurant
15,Downtown Toronto,5,Restaurant,Coffee Shop,Pub,Flower Shop,Café,Indian Restaurant,Pizza Place,Italian Restaurant,Bakery,Bank
16,Downtown Toronto,5,Coffee Shop,Café,Hotel,Restaurant,Steakhouse,American Restaurant,Deli / Bodega,Bar,Gastropub,Gym
17,Downtown Toronto,5,Japanese Restaurant,Coffee Shop,Gay Bar,Sushi Restaurant,Burger Joint,Restaurant,Pub,Mediterranean Restaurant,Men's Store,Bubble Tea Shop
